# --- Day 7: Camel Cards ---

Find the rank of every hand in your set. What are the total winnings?

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("input.txt", names=["hand", "bid"], delim_whitespace=True)
df

,hand,bid
0,8444T,864
1,6TK4Q,440
2,A5555,197
3,53353,712
4,6K68A,216
...,...,...
995,AK8A9,316
996,99992,129
997,48744,976
998,9A9KK,18


In [3]:
from collections import Counter
df["counts"] = df["hand"].apply(lambda x: Counter(list(x)))
df

,hand,bid,counts
0,8444T,864,"{'8': 1, '4': 3, 'T': 1}"
1,6TK4Q,440,"{'6': 1, 'T': 1, 'K': 1, '4': 1, 'Q': 1}"
2,A5555,197,"{'A': 1, '5': 4}"
3,53353,712,"{'5': 2, '3': 3}"
4,6K68A,216,"{'6': 2, 'K': 1, '8': 1, 'A': 1}"
...,...,...,...
995,AK8A9,316,"{'A': 2, 'K': 1, '8': 1, '9': 1}"
996,99992,129,"{'9': 4, '2': 1}"
997,48744,976,"{'4': 3, '8': 1, '7': 1}"
998,9A9KK,18,"{'9': 2, 'A': 1, 'K': 2}"


In [4]:
def hand(counts):
    
    # Five of Kind
    if 5 in counts.values():
        return 0
    
    # Four of a Kind
    if 4 in counts.values():
        return 1
    
    if 3 in counts.values():
        # Full House
        if 2 in counts.values():
            return 2
        # Three of a Kind
        else:
            return 3
    
    if 2 in counts.values():
        # Two Pair House
        if list(counts.values()).count(2) == 2:
            return 4
        # Pair
        else:
            return 5
        
    return 6

In [5]:
cards = list("AKQJT98765432")
cards

['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

In [6]:
df["rank"] = df["counts"].apply(hand)
df

,hand,bid,counts,rank
0,8444T,864,"{'8': 1, '4': 3, 'T': 1}",3
1,6TK4Q,440,"{'6': 1, 'T': 1, 'K': 1, '4': 1, 'Q': 1}",6
2,A5555,197,"{'A': 1, '5': 4}",1
3,53353,712,"{'5': 2, '3': 3}",2
4,6K68A,216,"{'6': 2, 'K': 1, '8': 1, 'A': 1}",5
...,...,...,...,...
995,AK8A9,316,"{'A': 2, 'K': 1, '8': 1, '9': 1}",5
996,99992,129,"{'9': 4, '2': 1}",1
997,48744,976,"{'4': 3, '8': 1, '7': 1}",3
998,9A9KK,18,"{'9': 2, 'A': 1, 'K': 2}",4


In [7]:
for i in range(5):
    df[f"card_rank_{i}"] = df["hand"].apply(lambda x: cards.index(x[i]))
df

,hand,bid,counts,rank,card_rank_0,card_rank_1,card_rank_2,card_rank_3,card_rank_4
0,8444T,864,"{'8': 1, '4': 3, 'T': 1}",3,6,10,10,10,4
1,6TK4Q,440,"{'6': 1, 'T': 1, 'K': 1, '4': 1, 'Q': 1}",6,8,4,1,10,2
2,A5555,197,"{'A': 1, '5': 4}",1,0,9,9,9,9
3,53353,712,"{'5': 2, '3': 3}",2,9,11,11,9,11
4,6K68A,216,"{'6': 2, 'K': 1, '8': 1, 'A': 1}",5,8,1,8,6,0
...,...,...,...,...,...,...,...,...,...
995,AK8A9,316,"{'A': 2, 'K': 1, '8': 1, '9': 1}",5,0,1,6,0,5
996,99992,129,"{'9': 4, '2': 1}",1,5,5,5,5,12
997,48744,976,"{'4': 3, '8': 1, '7': 1}",3,10,6,7,10,10
998,9A9KK,18,"{'9': 2, 'A': 1, 'K': 2}",4,5,0,5,1,1


In [8]:
df = df.sort_values(["rank", "card_rank_0", "card_rank_1", "card_rank_2", "card_rank_3", "card_rank_4"], ascending=False).reset_index(drop=True)
df.index += 1
df

,hand,bid,counts,rank,card_rank_0,card_rank_1,card_rank_2,card_rank_3,card_rank_4
1,2379T,372,"{'2': 1, '3': 1, '7': 1, '9': 1, 'T': 1}",6,12,11,7,5,4
2,23T6J,575,"{'2': 1, '3': 1, 'T': 1, '6': 1, 'J': 1}",6,12,11,4,8,3
3,23T85,83,"{'2': 1, '3': 1, 'T': 1, '8': 1, '5': 1}",6,12,11,4,6,9
4,258AT,901,"{'2': 1, '5': 1, '8': 1, 'A': 1, 'T': 1}",6,12,9,6,0,4
5,26Q97,527,"{'2': 1, '6': 1, 'Q': 1, '9': 1, '7': 1}",6,12,8,2,5,7
...,...,...,...,...,...,...,...,...,...
996,AKKKK,557,"{'A': 1, 'K': 4}",1,0,1,1,1,1
997,AA8AA,559,"{'A': 4, '8': 1}",1,0,0,6,0,0
998,AAA3A,637,"{'A': 4, '3': 1}",1,0,0,0,11,0
999,AAAQA,656,"{'A': 4, 'Q': 1}",1,0,0,0,2,0


In [9]:
df["earnings"] = df.apply(lambda x: x.name * x["bid"], axis=1)
df

,hand,bid,counts,rank,card_rank_0,card_rank_1,card_rank_2,card_rank_3,card_rank_4,earnings
1,2379T,372,"{'2': 1, '3': 1, '7': 1, '9': 1, 'T': 1}",6,12,11,7,5,4,372
2,23T6J,575,"{'2': 1, '3': 1, 'T': 1, '6': 1, 'J': 1}",6,12,11,4,8,3,1150
3,23T85,83,"{'2': 1, '3': 1, 'T': 1, '8': 1, '5': 1}",6,12,11,4,6,9,249
4,258AT,901,"{'2': 1, '5': 1, '8': 1, 'A': 1, 'T': 1}",6,12,9,6,0,4,3604
5,26Q97,527,"{'2': 1, '6': 1, 'Q': 1, '9': 1, '7': 1}",6,12,8,2,5,7,2635
...,...,...,...,...,...,...,...,...,...,...
996,AKKKK,557,"{'A': 1, 'K': 4}",1,0,1,1,1,1,554772
997,AA8AA,559,"{'A': 4, '8': 1}",1,0,0,6,0,0,557323
998,AAA3A,637,"{'A': 4, '3': 1}",1,0,0,0,11,0,635726
999,AAAQA,656,"{'A': 4, 'Q': 1}",1,0,0,0,2,0,655344


In [10]:
df["earnings"].sum()

251106089

# --- Part Two ---
Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [68]:
df = pd.read_csv("input.txt", names=["hand", "bid"], delim_whitespace=True)
df

,hand,bid
0,8444T,864
1,6TK4Q,440
2,A5555,197
3,53353,712
4,6K68A,216
...,...,...
995,AK8A9,316
996,99992,129
997,48744,976
998,9A9KK,18


In [69]:
df["counts"] = df["hand"].apply(lambda x: Counter(list(x)))
df

,hand,bid,counts
0,8444T,864,"{'8': 1, '4': 3, 'T': 1}"
1,6TK4Q,440,"{'6': 1, 'T': 1, 'K': 1, '4': 1, 'Q': 1}"
2,A5555,197,"{'A': 1, '5': 4}"
3,53353,712,"{'5': 2, '3': 3}"
4,6K68A,216,"{'6': 2, 'K': 1, '8': 1, 'A': 1}"
...,...,...,...
995,AK8A9,316,"{'A': 2, 'K': 1, '8': 1, '9': 1}"
996,99992,129,"{'9': 4, '2': 1}"
997,48744,976,"{'4': 3, '8': 1, '7': 1}"
998,9A9KK,18,"{'9': 2, 'A': 1, 'K': 2}"


In [70]:
cards = list("AKQT98765432J")
cards

['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']

In [71]:
import itertools
import copy

def jokerize(counts):
    
    rank = hand(counts)
    
    if "J" not in counts:
        return rank
    
    # Evaluate each combination of new cards
    for combination in itertools.product(*[cards for _ in range(counts["J"])]):
        
        counts_tmp = copy.deepcopy(counts)
        for new_ in combination:
            
            try:
                counts_tmp[new_] += 1
            except KeyError:
                counts_tmp[new_] = 1
        
        counts_tmp["J"] = 0
        if hand(counts_tmp) < rank:
            rank = hand(counts_tmp)
        
    return rank


In [72]:
df["rank"] = df["counts"].apply(lambda x: jokerize(x))
df

,hand,bid,counts,rank
0,8444T,864,"{'8': 1, '4': 3, 'T': 1}",3
1,6TK4Q,440,"{'6': 1, 'T': 1, 'K': 1, '4': 1, 'Q': 1}",6
2,A5555,197,"{'A': 1, '5': 4}",1
3,53353,712,"{'5': 2, '3': 3}",2
4,6K68A,216,"{'6': 2, 'K': 1, '8': 1, 'A': 1}",5
...,...,...,...,...
995,AK8A9,316,"{'A': 2, 'K': 1, '8': 1, '9': 1}",5
996,99992,129,"{'9': 4, '2': 1}",1
997,48744,976,"{'4': 3, '8': 1, '7': 1}",3
998,9A9KK,18,"{'9': 2, 'A': 1, 'K': 2}",4


In [73]:
for i in range(5):
    df[f"card_rank_{i}"] = df["hand"].apply(lambda x: cards.index(x[i]))
df

,hand,bid,counts,rank,card_rank_0,card_rank_1,card_rank_2,card_rank_3,card_rank_4
0,8444T,864,"{'8': 1, '4': 3, 'T': 1}",3,5,9,9,9,3
1,6TK4Q,440,"{'6': 1, 'T': 1, 'K': 1, '4': 1, 'Q': 1}",6,7,3,1,9,2
2,A5555,197,"{'A': 1, '5': 4}",1,0,8,8,8,8
3,53353,712,"{'5': 2, '3': 3}",2,8,10,10,8,10
4,6K68A,216,"{'6': 2, 'K': 1, '8': 1, 'A': 1}",5,7,1,7,5,0
...,...,...,...,...,...,...,...,...,...
995,AK8A9,316,"{'A': 2, 'K': 1, '8': 1, '9': 1}",5,0,1,5,0,4
996,99992,129,"{'9': 4, '2': 1}",1,4,4,4,4,11
997,48744,976,"{'4': 3, '8': 1, '7': 1}",3,9,5,6,9,9
998,9A9KK,18,"{'9': 2, 'A': 1, 'K': 2}",4,4,0,4,1,1


In [74]:
df = df.sort_values(["rank", "card_rank_0", "card_rank_1", "card_rank_2", "card_rank_3", "card_rank_4"], ascending=False).reset_index(drop=True)
df.index += 1
df

,hand,bid,counts,rank,card_rank_0,card_rank_1,card_rank_2,card_rank_3,card_rank_4
1,2379T,372,"{'2': 1, '3': 1, '7': 1, '9': 1, 'T': 1}",6,11,10,6,4,3
2,23T85,83,"{'2': 1, '3': 1, 'T': 1, '8': 1, '5': 1}",6,11,10,3,5,8
3,258AT,901,"{'2': 1, '5': 1, '8': 1, 'A': 1, 'T': 1}",6,11,8,5,0,3
4,26Q97,527,"{'2': 1, '6': 1, 'Q': 1, '9': 1, '7': 1}",6,11,7,2,4,6
5,293A8,762,"{'2': 1, '9': 1, '3': 1, 'A': 1, '8': 1}",6,11,4,10,0,5
...,...,...,...,...,...,...,...,...,...
996,TTJJT,617,"{'T': 3, 'J': 2}",0,3,3,12,12,3
997,QQQJQ,781,"{'Q': 4, 'J': 1}",0,2,2,2,12,2
998,KKJKJ,86,"{'K': 3, 'J': 2}",0,1,1,12,1,12
999,KKKKJ,357,"{'K': 4, 'J': 1}",0,1,1,1,1,12


In [75]:
df["earnings"] = df.apply(lambda x: x.name * x["bid"], axis=1)
df

,hand,bid,counts,rank,card_rank_0,card_rank_1,card_rank_2,card_rank_3,card_rank_4,earnings
1,2379T,372,"{'2': 1, '3': 1, '7': 1, '9': 1, 'T': 1}",6,11,10,6,4,3,372
2,23T85,83,"{'2': 1, '3': 1, 'T': 1, '8': 1, '5': 1}",6,11,10,3,5,8,166
3,258AT,901,"{'2': 1, '5': 1, '8': 1, 'A': 1, 'T': 1}",6,11,8,5,0,3,2703
4,26Q97,527,"{'2': 1, '6': 1, 'Q': 1, '9': 1, '7': 1}",6,11,7,2,4,6,2108
5,293A8,762,"{'2': 1, '9': 1, '3': 1, 'A': 1, '8': 1}",6,11,4,10,0,5,3810
...,...,...,...,...,...,...,...,...,...,...
996,TTJJT,617,"{'T': 3, 'J': 2}",0,3,3,12,12,3,614532
997,QQQJQ,781,"{'Q': 4, 'J': 1}",0,2,2,2,12,2,778657
998,KKJKJ,86,"{'K': 3, 'J': 2}",0,1,1,12,1,12,85828
999,KKKKJ,357,"{'K': 4, 'J': 1}",0,1,1,1,1,12,356643


In [76]:
df["earnings"].sum()

249620106